# Week 3 Exercise

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

import pandas as pd
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL

In [3]:
PROJ_ROOT = os.pardir
src_dir = os.path.join(PROJ_ROOT, "src")
sys.path.append(src_dir)

In [4]:
%aimport sql_utils
from sql_utils import compare_outputs

## About

### Objective

We have a beta version of the website available and have opened it for use by a small number of customers. Next week we plan to increase this number [by a factor of four].

Once we expand the beta version, we expect the table to grow very quickly. While it is still fairly small, we need to develop a query to measure the impact of the changes to our search algorithm. Please create a query and review the query profile to ensure that the query will be efficient once the activity increases.

We want to create a daily report to track

1. Total unique sessions
2. The average length of sessions in seconds
3. The average number of searches completed before displaying a recipe 
4. The ID of the recipe that was most viewed 

In addition to your query, please submit a short description of what you determined from the query profile and how you structured your query to plan for a higher volume of events once the website traffic increases.

### Overview of Best Practices Used

Below are some Snowflake best practices that were used to optimize query performance here

#### [Query Optimization](https://www.chaosgenius.io/blog/snowflake-query-tuning-part1/)
1. Choose a larger warehouse size and compare the performance gain of a larger warehouse to the extra cost
   - admin permissions required
   - not controllable here
   - <span style="color:red">this was not used here</span>
2. Use result caching
   - by default, the result cache expires after 24 hours
     - this could be leveraged during production by pre-running the report query; subsequent runs of the report within the next 24 hours can leverage the cached result to get fast execution times
     - we don't know if the report must be run by multiple stakeholders in production or if result caching can be leveraged in production
   - not known if this is useful here
   - <span style="color:red">this was not used here</span>
3. Use materialized views
   - these are beneficial when the view's data does not change frequently but is used often
     - the report is generated daily
     - we don't know if the report must be run by multiple stakeholders in production and how often they will do so
   - not known if this is useful here
   - <span style="color:red">this was not used here</span>
4. Use data clustering and micro partitioning to speed up queries
   - the `events.warehouse_activity` table does not support clustering
   - not relevant here
   - <span style="color:red">this was not used here</span>
5. Use Snowflake query acceleration
   - admin permissions required
   - not controllable here
   - <span style="color:red">this was not used here</span>
6. Use Snowflake query profiler
   - <span style="color:green">this was used here</span>

#### [Query Tuning](https://www.chaosgenius.io/blog/snowflake-query-tuning-2/)
1. Select only required columns (also called [projection pushdown](https://trino.io/docs/current/optimizer/pushdown.html#projection-pushdown))
   - <span style="color:green">this was used here</span>
2. Optimize sorting (reduce calls to `ORDER BY`)
   - here, the order of query results is not speicfied
     - we will **assume** that the report will want the above four metrics in chronological order for readability, so *<u>only</u>* the final query will be sorted by session date
   - <span style="color:green">this was used here</span>
3. Using `JOIN`s instead of sub-queries
   - this will not be used here
   - [some SQL style guides prefer to avoid this approach](https://github.com/brooklyn-data/co/blob/main/sql_style_guide.md#when-inner-joining-put-filter-conditions-in-the-where-clause-instead-of-the-join-clause) and recommend placing filter conditions in the `WHERE` clause (similar to [predicate pushdown](https://trino.io/docs/current/optimizer/pushdown.html#projection-pushdown))
     - predicate pushdown will be used here
   - <span style="color:green">this was used here</span>

## Create SQL Engine

In [5]:
engine = create_engine(
    URL(
        drivername="driver",
        account=os.getenv("UPLIMIT_SNOWFLAKE_ACCOUNT"),
        user=os.getenv("UPLIMIT_SNOWFLAKE_USER"),
        password=os.getenv("UPLIMIT_SNOWFLAKE_PASS"),
        warehouse=os.getenv("UPLIMIT_SNOWFLAKE_WAREHOUSE"),
        database=os.getenv("UPLIMIT_SNOWFLAKE_DB_NAME"),
        role=os.getenv("UPLIMIT_SNOWFLAKE_ROLE"),
        timezone=os.getenv("MY_TIMEZONE"),
    ),
    connect_args=dict(session_parameters={"USE_CACHED_RESULT": False}),
)

## Connect

Load SQL extension

In [6]:
%load_ext sql

Connect to Uplimit's Snowflake instance

In [7]:
%sql engine --alias connection

Display all rows from the query output

In [8]:
%config SqlMagic.displaylimit = None

displaylimit: Value None will be treated as 0 (no limit)

Show warehouse

In [9]:
%%sql result = <<
SHOW WAREHOUSES LIKE 'TRANSFORM'

Running query in 'connection'

1 rows affected.

name,state,type,size,running,queued,is_default,is_current,auto_suspend,auto_resume,available,provisioning,quiescing,other,created_on,resumed_on,updated_on,owner,comment,resource_monitor,actives,pendings,failed,suspended,uuid,budget,owner_role_type
TRANSFORM,SUSPENDED,STANDARD,X-Small,0,0,Y,Y,1,true,,,,,2022-12-28 08:05:52.258000-05:00,2024-05-13 16:23:41.723000-04:00,2024-05-13 16:23:41.723000-04:00,ACCOUNTADMIN,,null,0,0,0,1,55437832,None,ROLE


## Queries

### Check for Use of Clustering in `events.website_activity` Table

As shown in the two hilighted columns below, the `website_activity` table does not have clustering enabled.

#### Using `information_schema.tables`

In [10]:
%%sql result <<
SELECT table_Schema,
       table_name,
       row_count,
       clustering_key,
       auto_clustering_on
FROM information_schema.tables
WHERE LOWER(table_name) = 'website_activity'

Running query in 'connection'

1 rows affected.

In [11]:
(
    result
    .DataFrame()
    .style
    .set_properties(
        subset=['clustering_key', 'auto_clustering_on'],
        **{'background-color': 'yellow'}
    )
    .hide(subset=None, axis=0, level=None, names=False)
)

table_schema,table_name,row_count,clustering_key,auto_clustering_on
EVENTS,WEBSITE_ACTIVITY,348,None,NO


#### Using `SHOW TABLES`

In [12]:
%%sql result <<
SHOW TABLES LIKE 'WEBSITE_ACTIVITY'

Running query in 'connection'

1 rows affected.

In [13]:
(
    result
    .DataFrame()
    [['schema_name', 'name', 'rows', 'cluster_by', 'automatic_clustering']]
    .style
    .set_properties(
        subset=['cluster_by', 'automatic_clustering'],
        **{'background-color': 'yellow'}
    )
    .hide(subset=None, axis=0, level=None, names=False)
)

schema_name,name,rows,cluster_by,automatic_clustering
EVENTS,WEBSITE_ACTIVITY,348,,OFF


### Using Single CTE with Session Stats and Length

The overall query to generate the daily report by using a single CTE with both session statistics and session lengths by following the five steps listed below. Each step corresponds to a CTE. Only the first CTE queries the raw events data from the `events.website_activity` table.

1. perform the following operations on the events level data
   - use `PARSE_JSON` to convert (a) <span style="color:blue">last event</span> and (b) all <span style="color:magenta">events</span> in the `event_details` column [into the `VARIANT` datatype that holds `JSON` information](https://docs.snowflake.com/en/sql-reference/functions/parse_json) whose attributes be accessed later
   - extract session date from the `event_timestamp` column as the date on which the session started
   - exclude columns that are not used in downstream CTEs of the overall query
     - `user_id`
     - `event_id`
2. get the most viewed `recipe_id` every day
   - (a) get all sessions in which the <span style="color:blue">last event</span> was viewing a recipe
   - (b) for all matching sessions from (a), use `GROUP BY` get the
     - (i) `recipe_id`
     - (ii) number of <span style="color:blue">events in which a recipe was viewed</span> per day
       - this is the number of daily views per recipe, which will be used to rank the recipes
   - (c) get the `recipe_id` with the most daily views
     - use a `RANK()` window function to assign ranks to the `recipe_id`s based on their number of daily views
     - use `QUALIFY` to get the `recipe_ids` for which `RANK() OVER(...) = 1`
3. Summarize each session by getting the following stats per session per date
   - (a) indicate if the session ended with a view of a recipe (i.e. if the <span style="color:blue">last event</span> in a session was to view a recipe)
   - (b) count the number of <span style="color:green">events in which a search performed</span> during the session
     - this will be used to answer question 3
   - (c) calculate the session length in seconds using `TIMESTAMPDIFF('second', ...)`
4. From the output of step 3., get the following stats per date
   - (a) number of unique sessions from 3. (a)
   - (b) average session length from 3. (c)
   - (c) average number of searches performed before viewing recipe
     - for sessions in which a recipe was viewed from 3. (a), this is the average of the number of daily views during those sessions which was found in 3. (b)
5. Combine daily session summary from step 4. and most daily viewed recipe(s) from step 2.
   - (a) perform a `LEFT JOIN` using the session date column since both steps 2. and 4. are aggregated by date
   - (b) in the `daily_session_summary` CTE, session date was extracted as the date on which the session started. Some sessions can be spread out across two days. Such sessions should only have been counted on the first day of the session and not on the second day. Also, session length was recorded on the first date since `session_date` was recorded on the date on which the session started, so the session length should been excluded from the second date. In summary, such multi-day sessions should not have been (i) counted on both days and (ii) used to calculate session length on the second day. However, such logic has not been implemented earlier in the query. So, these rows must now be filtered out using `WHERE session_length > 0`.
   - (c) the output of step 2. contains some dates on which multiple recipes were tied as the most viewed recipe since they had the same number of views. For these rows, concatenate the `recipe_id`s into a comma-separated string on a single row using `GROUP BY`+`LISTAGG`. Based on [SQL order of operations](https://www.sisense.com/blog/sql-query-order-of-operations/), the `JOIN` is executed before the `GROUP BY`. For this reason, a `LEFT JOIN` is needed so that no rows are lost if multiple recipes were tied for the most daily views.
   - (d) for readability, sort the result in chronological order using the session date column

In [14]:
%%sql result = <<
/* step 1. get JSON of the last event and all events performed per session */
WITH session_last_event_all_events_json AS (
    SELECT session_id,
           event_timestamp,
           /* take the start date of the session as the session date */
           MIN(TO_DATE(event_timestamp)) OVER(PARTITION BY session_id) AS session_date,
           /* get JSON of last event */
           LAST_VALUE(PARSE_JSON(event_details)) OVER(
               PARTITION BY session_id
               ORDER BY event_timestamp
           ) AS last_event_json,
           /* get JSON of all events */
           PARSE_JSON(event_details) AS event_json
    FROM events.website_activity
),
/* step 2. get the most viewed recipe each day
   (a) get sessions in which the last event was viewing a recipe
   (b) for the (last) event in the sessions from 2. (a), get (i) recipe_id and
   (ii) get the number of times the recipe was viewed daily
   (c) get the recipe_id with the most daily views using QUALIFY */
most_viewed_recipe_daily AS (
    SELECT session_date,
           /* step 2. (b) (i) get recipe_id */
           last_event_json:recipe_id AS recipe_id
    FROM session_last_event_all_events_json
    /* step 2. (a) get sessions where the last event was to view a recipe */
    WHERE last_event_json:event = 'view_recipe'
    /* step 2. (b) get events in which the recipe was viewed */
    AND event_json:event = 'view_recipe'
    GROUP BY ALL
    /* step 2. (c) get the recipe_id with the most daily views */
    QUALIFY RANK() OVER(PARTITION BY session_date ORDER BY COUNT(*) DESC) = 1
),
/* step 3. summarize each session by getting the following stats
   (a) indicate if it ended with a user viewing a recpie
   (b) count number of searches performed during the session
   (c) get session length */
session_summary AS (
    SELECT session_date,
           session_id,
           /* step 3. (a) indicate if session ended with viewing recipe */
           (
               CASE
                   WHEN last_event_json:event = 'view_recipe'
                   THEN TRUE
                   ELSE FALSE
               END
           ) AS has_viewed_recipe,
           /* step 3. (b) count number of searches performed per session */
           SUM(
               CASE
                   WHEN event_json:event = 'search'
                   THEN 1
                   ELSE 0
               END
           ) AS num_searches,
           /* step 3. (c) get session length */
           TIMESTAMPDIFF(
               'second', MIN(event_timestamp), MAX(event_timestamp)
           ) AS session_length
    FROM session_last_event_all_events_json
    GROUP BY ALL
),
/* step 4. aggregate stats per day to get the following
   (a) number of sessions
   (b) average session length in seconds
   (c) average number of searches before viewing a recipe */
daily_session_summary AS (
    SELECT session_date,
           /* step 4. (a) number of daily sessions */
           COUNT(session_id) AS num_sessions,
           /* step 4. (b) average daily session length */
           AVG(session_length) AS avg_session_length_seconds,
           /* step 4. (c) average number of searches before viewing recipe */
           AVG(
               CASE
                   WHEN has_viewed_recipe = TRUE
                   THEN num_searches
                   ELSE NULL
               END
           ) AS avg_num_searches_before_viewing_recipe
    FROM session_summary
    GROUP BY ALL
),
/* step 5. combine daily session summary and most daily viewed recipe(s)
   (a) perform LEFT JOIN between recipe(s) with the most daily views and
   daily session summary
   (b) handle multi-day sessions using WHERE avg_session_length_seconds > 0
   (c) concatenate most-viewed recipes into comma-delimited string, using
   GROUP BY+LISTAGG
   (d) sort result by session date in chronological order for readability */
daily_report_summary_sessions_recipe AS (
    SELECT * EXCLUDE(most_viewed_recipe),
           /* clean the most_viewed_recipe ID(s) per day */
           REPLACE(most_viewed_recipe, '"', '') AS most_viewed_recipe
    FROM (
        SELECT ds.session_date,
               ds.num_sessions,
               ds.avg_session_length_seconds,
               ds.avg_num_searches_before_viewing_recipe,
               /* step 5. (b) concatenate ties beteween most-viewed recipes into
                  single row */
               LISTAGG(dv.recipe_id, ', ') AS most_viewed_recipe
        FROM daily_session_summary AS ds
        /* step 5. (a) combine session summary and most-viewed receipe each day */
        LEFT JOIN most_viewed_recipe_daily AS dv USING (session_date)
        /* step 5. (b) exclude sessions which have been counted on previous day and so
           have a session length of zero */
        WHERE avg_session_length_seconds > 0
        GROUP BY ALL
        /* step 5. (c) sort result in chronological order for readability */
        ORDER BY ds.session_date
    )
)
SELECT *
FROM daily_report_summary_sessions_recipe

Running query in 'connection'

19 rows affected.

session_date,num_sessions,avg_session_length_seconds,avg_num_searches_before_viewing_recipe,most_viewed_recipe
2023-01-24,5,250.600000,3.000000,"5b0a19ca-00e8-4a6e-8e60-8a87f09e6b0f, 44dcd777-5b10-41e2-90df-4dca4b696971"
2023-01-26,4,99.250000,2.000000,44dcd777-5b10-41e2-90df-4dca4b696971
2023-01-27,5,171.200000,3.000000,5b0a19ca-00e8-4a6e-8e60-8a87f09e6b0f
2023-01-28,6,219.833333,3.000000,"5b0a19ca-00e8-4a6e-8e60-8a87f09e6b0f, 44dcd777-5b10-41e2-90df-4dca4b696971"
2023-01-29,1,427.000000,2.000000,44dcd777-5b10-41e2-90df-4dca4b696971
2023-01-30,3,284.000000,2.000000,44dcd777-5b10-41e2-90df-4dca4b696971
2023-01-31,10,228.300000,2.200000,44dcd777-5b10-41e2-90df-4dca4b696971
2023-02-01,7,15.000000,1.000000,44dcd777-5b10-41e2-90df-4dca4b696971
2023-02-02,25,38.880000,1.000000,44dcd777-5b10-41e2-90df-4dca4b696971
2023-02-03,27,55.259259,1.000000,44dcd777-5b10-41e2-90df-4dca4b696971


**Notes**

1. The above query produces the same number of sessions found previously during [exploratory data analysis](https://nbviewer.org/github/edesz/adv-sql/blob/main/07_week3_exercise_eda.ipynb), which can be seen from the output of the following query
   ```sql
   .
   .
   .
   SELECT SUM(num_sessions) AS total_num_sessions
   FROM daily_report_summary_sessions_recipe
   ```

## Inspection of Queries

In addition to your query, please submit

1. a short description of what you determined from the query profile
2. how you structured your query to plan for a higher volume of events once the website traffic increases.

### Using Query Profile to Check for Query Problems

#### Use the **Query Profile** From Snowsight GUI To Understand Query Execution Plan

The execution plan from the Snowflake **Query Profile** is available on slides 2 and 3 from [here](https://docs.google.com/presentation/d/1Gw77tUlzvMMZtpwWIFg-5KZ_o4ZA4V0livaAJBjQz1M/edit?usp=sharing).

**Observations**

1. The `WithReference` operator appears twice in the query execution plan. This is expected since the first CTE is referenced by the (a) `most_viewed_recipe_daily` and (b) `session_summary` CTEs.
2. The `Sort` operator is expensive and only appears once. This is expected since `ORDER BY` was only applied on the final result which was aggregated daily.
3. `JoinFilter` is used to identify rows that do not match the `JOIN` condition. Here, a `LEFT JOIN` is used where there are more rows on the LHS, so there are no mismatched rows to filter out. In the query execution plan, this is labeled as <span style="color:orange">`LEFT JOIN` is used so this has no impact</span>.
4. A `Filter` is applied after a `TableScan` once. In this scenario, the `TableScan` is performing the filtering so the `Filter` step has no impact. Also, a `WHERE` clause is not used in the main CTE. For this reason, the input and output row counts of the filter are the same after the `Filter` operator. In the query execution plan diagram, `Filter` is labeled as <span style="color:#d4628d">does not do anything</span>.
5. From a high-level view of the query execution plan on slide 2
   - the two types of metrics to be calculated (three session-based metrics and one metric for recipe views) are separated and are shown in <span style="color:blue">blue</span> and <span style="color:#df03db">dark pink</span>. This is expected since this separation was necessary since a window function function was used to find the most viewed daily recipe
     - `session_summary` and `daily_session_summary` calculate the three daily session-based metrics. These CTEs are shown in the <span style="color:blue">blue box</span>. They do not need to rank recipes based on views, so they do not need to apply the `RANK()` window function.
     - `most_viewed_recipe_daily` is the only CTE that is concerned about ranking recipes based on daily views. These CTEs are shown in the <span style="color:#df03db">dark pink box</span>.
6. From the zoomed verson of the execution plan on slide 3, the two separate daily aggregations are `LEFT JOIN`ed. The number of rows after the `JOIN` is larger than before the `JOIN` since multiple recipes are tied for most views on some days. This is labeled on slide 3 as *<span style="color:#78339a">More rows after LEFT JOIN due to ties in RANK()</span>*. It is reassuring that after concatenating ties into the same row using `LISTAGG`, the number of rows matches those in the CTE on the LHS of the `LEFT JOIN` and there is one row per date.
7. The query profile verifies findings from Exploratory Data Analysis (EDA) and are labeled on slide 3 as *<span style="color:#7a3d18">verifies finding from EDA</span>*
   - in `most_viewed_recipe_daily`, 55 rows are produced which matches 55 recipe views found during [EDA](https://nbviewer.org/github/edesz/adv-sql/blob/main/07_week3_exercise_eda.ipynb)
   - in `session_summary`, 178 rows are produced which the total unique sessions found during [EDA](https://nbviewer.org/github/edesz/adv-sql/blob/main/07_week3_exercise_eda.ipynb)
8. In the first CTE, the `WindowFunction` node (shown in the <span style="color:green">green box</span> on slide 3) is the most expensive node of the entire query. This is expected for two reasons
   - since two window functions were used and they were both applied to all events in the table. It was necessary to read all events from the table in order to calculate session length for the second report metric (average session length)
   - the window function contains a nested `PARSE_JSON`, which is also applied to each row since we need to extract the attributes of the `event_details` column for all events in the table so they could be accessed in CTEs
9. There are a lot of processing operations being performed such as aggregations, window functions and `JOIN`s. So, from the **Profile Overview**, it is not surprising that processing time dominates the overall time required to execute the query.

#### Use `get_query_operator_stats()` To Check for Four Common Types of Query Problems

The query will be scalable as more users are added to the platform if it does not contain [four common types of query problems](https://docs.snowflake.com/en/user-guide/ui-query-profile#common-query-problems-identified-by-query-profile)

1. Check for presence of exploding `JOIN`s
   - more rows are produced after the `JOIN` than before the `JOIN`
   - if not performing a `CROSS JOIN` (cartesian product), then this outcome indicates an error in how the `JOIN` was performed
   - in the
     - [Query execution plan diagram](https://docs.snowflake.com/en/user-guide/ui-snowsight-activity#snowsight-activity-query-execution-plan)
       - an exploding `JOIN` appears if the number of records produced by a `JOIN` operator is larger than the number of records going into that operator
     - output of the [`get_query_operator_stats()` function](https://docs.snowflake.com/en/sql-reference/functions/get_query_operator_stats)
       - an exploding `JOIN` appears if the following conditions are met
         - ratio of `output_rows/input_rows` is greater than 1
2. Check for use of `UNION` when `UNION ALL` would have been sufficient to concatenate CTEs or tables
   - `UNION` drops duplicates so **`UNION ALL` is more performant** since it does not drop duplicates
   - in the
     - Query execution plan diagram
       - [these occurrences appear as a `UnionAll` operator combined with an extra `Aggregate` operator](https://docs.snowflake.com/en/user-guide/ui-snowsight-activity#union-without-all) which is a redundant operation
     - output of the `get_query_operator_stats()` function
       - these occurrences appear if the following conditions are met
         ```sql
         operator_type = 'UnionAll'
         AND LAG(operator_type) OVER (ORDER BY operator_id) = 'Aggregate'
         ```
3. Check for inefficient use of pruning if the query uses a `WHERE` clause to filter out a significant amount of data
   - **efficient pruning is more performant** than reading the full table since some parts of a table might not need to be read when the filter is applied
   - in the
     - Query execution plan diagram
       - [inefficient pruning appears if *Statistics* > *Partitions Scanned* are less than *Statistics* > *Total Partitions*](https://docs.snowflake.com/en/user-guide/ui-snowsight-activity#inefficient-pruning)
     - output of the `get_query_operator_stats()` function
       - inefficient pruning appears if the following conditions are met
         - `partitions_total = 1`
           - the ratio `partitions_scanned/partitions_total` is equal to 1
         - `partitions_total > 1`
           - the ratio `partitions_scanned/partitions_total` is greater than some threshold such as 0.8 (or 80%) which would suggest pruning has not been very effective since 80% of partitions have to be scanned
         - the table's clustering key is not `NULL`
4. Check for queries whose intermediate results are [too large to fit into memory, which causes spilling](https://docs.snowflake.com/en/user-guide/ui-snowsight-activity#queries-too-large-to-fit-in-memory) to the local or remote disk
   - it **is more performant** if (a) **spilling is eliminated**, or (b) **spilling only occurs to local disk** [than spilling to remote disk](https://community.snowflake.com/s/article/Performance-impact-from-local-and-remote-disk-spilling)
   - in the
     - Query execution plan diagram
       - spilling to the
         - local disk
           - appears as *Spilling* > *Local Disk I/O*
         - remote disk
           - appears as *Spilling* > *Remote Disk I/O*
     - output of the `get_query_operator_stats()` function
       - spilling to the
         - local disk
           - appears if `bytes_spilled_local_storage>0`
         - remote disk
           - appears if `bytes_spilled_remote_storage>0`

Here, these checks will be implemented using the following [operator](https://docs.snowflake.com/en/sql-reference/operators) fields from the output of the `get_query_operator_stats()` function

1. `step_id`
   - identifier for step
2. columns for informational purposes
   - `table_name` (manually added, not from `get_query_operator_stats()`)
     - table name
   - `clustering_key`
     - clustering key, if present (manually added, not from `get_query_operator_stats()`)
   - `overall_pct` (highest non-zero values in this column are the same as **Most Expensive Notes** from GUI)
     - percentage of total execution time spent on each node
   - `bytes_scanned_mb` (total of this column is the same as **Bytes scanned** from Snowsight GUI)
     - Megabytes of data in the table that is scanned by the query
3. columns needed to check for exploding `JOIN`s (check 1/4)
   - `input_rows`
     - number of rows input to the operator
   - `output_rows`
       - number of rows returned by the operator
4. columns to check for use of `UNION` instead of `UNION ALL` (check 2/4)
   - also kept as operator metadata
   - `operator_id`
     - operator identifier
   - `operator_type`
     - type of operator (`TableScan`, `Filter`, `Aggregate`, `Result`)
5. columns needed to check for inefficient pruning by the `Filter` operator (check 3/4)
   - `partitions_total` (same as **Partitions total** from Snowsight GUI)
     - number of partitions in a table
   - `partitions_scanned` (same as **Partitions scanned** from Snowsight GUI)
     - number of partitions scanned by the operator
6. columns needed to check for queries that are too big to fit in memory and spill to disk (check 4/4)
   - `bytes_spilled_local_storage` (same as **Local Disk I/O** from Snowsight GUI)
     - data volume spilled by the operator to local disk, in bytes
     - also returned for informational purposes
   - `bytes_spilled_remote_storage` (same as **Remote Disk I/O** from Snowsight GUI)
     - data volume spilled by the operator to remote disk, in bytes
     - also returned for informational purposes

In [15]:
%%sql result = <<
/* step 1. get operator fields for the query */
WITH operator_stats AS (
    SELECT query_id,
           'WEBSITE_ACTIVITY' AS table_name,
           step_id,
           /* get fields to check for exploding JOINs */
           operator_statistics:input_rows AS input_rows,
           operator_statistics:output_rows::int AS output_rows,
           /* get fields to check for use of UNION instead of UNION ALL */
           operator_id,
           operator_type,
           /* get fields to check for inefficient pruning */
           operator_statistics:pruning.partitions_total::int AS partitions_total,
           operator_statistics:pruning.partitions_scanned::int AS partitions_scanned,
           partitions_scanned/partitions_total::float AS partition_scan_ratio,
           /* get fields to check for queries too large to fit into memory */
           operator_statistics:spilling:bytes_spilled_local_storage::int AS bytes_spilled_local_storage,
           operator_statistics:spilling:bytes_spilled_remote_storage::int AS bytes_spilled_remote_storage,
           /* (optional) TBD */
           100*(execution_time_breakdown:overall_percentage)::float AS overall_pct,
           operator_statistics:io.bytes_scanned/1000000::float AS bytes_scanned_mb
    FROM TABLE(get_query_operator_stats('01b44d8e-0001-d5e5-0003-4dea007f7eca'))
),
/* step 2. check for four types of query problems */
query_problem_checks AS (
    SELECT step_id,
           /* return fields for informational purposes */
           ist.table_name,
           ist.clustering_key,
           overall_pct,
           bytes_spilled_local_storage,
           bytes_spilled_remote_storage,
           bytes_scanned_mb,
           /* keep metadata for operators */
           operator_id,
           operator_type,
           /* check for exploding JOINs */
           (
               CASE
                   WHEN input_rows>0
                   THEN output_rows/input_rows
                   ELSE 0
               END
           ) AS row_multiple,
           CASE WHEN row_multiple > 1 THEN 1 ELSE 0 END AS causes_exploding_join,
           /* check for use of UNION instead of UNION ALL */
           (
               CASE
                   WHEN
                        operator_type = 'UnionAll'
                        AND LAG(operator_type) OVER (ORDER BY operator_id) = 'Aggregate'
                   THEN 1
                   ELSE 0
               END
           ) AS uses_union_without_all,
           /* check for inefficient pruning */
           (
               CASE
                   WHEN
                       (
                           (
                               partition_scan_ratio >= 0.8
                               AND partitions_total > 1
                           ) OR (
                               partition_scan_ratio = 1
                               AND partitions_total = 1
                           )
                       )
                       AND operator_type = 'TableScan'
                       AND clustering_key IS NOT NULL
                   THEN 1
                   ELSE 0
               END
           ) AS uses_inefficient_pruning,
           /* check for query is too large to fit in memory */
           CASE
               WHEN (
                   bytes_spilled_local_storage>0
                   OR bytes_spilled_remote_storage>0
               )
               THEN 1
               ELSE 0
           END AS is_to_large_to_fit_in_memory
    FROM operator_stats
    LEFT JOIN (
        SELECT table_name,
               clustering_key 
        FROM information_schema.tables
        WHERE LOWER(table_name) = 'website_activity'
    ) AS ist USING (table_name)
    ORDER BY query_id, step_id, operator_id
)
SELECT *
FROM query_problem_checks

Running query in 'connection'

17 rows affected.

step_id,table_name,clustering_key,overall_pct,bytes_spilled_local_storage,bytes_spilled_remote_storage,bytes_scanned_mb,operator_id,operator_type,row_multiple,causes_exploding_join,uses_union_without_all,uses_inefficient_pruning,is_to_large_to_fit_in_memory
1,WEBSITE_ACTIVITY,None,2.941176470588235,None,None,None,0,Result,None,0,0,0,0
1,WEBSITE_ACTIVITY,None,5.88235294117647,None,None,None,1,Sort,1.0,0,0,0,0
1,WEBSITE_ACTIVITY,None,11.76470588235294,None,None,None,2,Aggregate,0.8333333333333334,0,0,0,0
1,WEBSITE_ACTIVITY,None,23.52941176470588,None,None,None,3,Join,0.5581395348837209,0,0,0,0
1,WEBSITE_ACTIVITY,None,17.647058823529413,None,None,None,4,Aggregate,0.11235955056179775,0,0,0,0
1,WEBSITE_ACTIVITY,None,0.0,None,None,None,5,Aggregate,0.5114942528735632,0,0,0,0
1,WEBSITE_ACTIVITY,None,0.0,None,None,None,6,WithReference,1.0,0,0,0,0
1,WEBSITE_ACTIVITY,None,0.0,None,None,None,7,WithClause,1.0,0,0,0,0
1,WEBSITE_ACTIVITY,None,29.411764705882355,None,None,None,8,WindowFunction,1.0,0,0,0,0
1,WEBSITE_ACTIVITY,None,0.0,None,None,0.016384,9,TableScan,0.0,0,0,0,0


**Notes**

1. The answer to each question is provided in the last four columns of this output
   - for check 1, see the `causes_exploding_join` column
   - for check 2, see `uses_union_without_all`
   - for check 3, see `uses_inefficient_pruning`
   - for check 4, see `is_to_large_to_fit_in_memory`

**Observations**

1. Check 2 for the use of `UNION` instead of `UNION ALL` in an operator is not relevant since neither was used here.
2. It is reassuring that none of the operators result in an exploding join (check 1, from the `causes_exploding_join` column) or return intermediate a resultset that is too large to fit in memory (check 4, from the `is_to_large_to_fit_in_memory` column). This will help with the scalability of the query when the user base grows from 200 to 5,000.
3. Check 3 for the presence of inefficient pruning in an operator is not relevant since the table does not use clustering (see the `clustering_key` column).

#### Summary of Findings Determined from the Snowflake Query Profile

**From the Snowsight Query Profile GUI**

1. The `WithReference` operator appears twice in the query execution plan. This is expected since the first CTE is referenced by the (a) `most_viewed_recipe_daily` and (b) `session_summary` CTEs.
2. The `Sort` operator is expensive and only appears once. This is expected since `ORDER BY` was only applied on the final result which was aggregated daily.
3. A `Filter` is applied after a `TableScan` once. In this scenario, the `TableScan` is performing the filtering so the `Filter` step has no impact. Also, a `WHERE` clause is not used in the main CTE. For this reason, the input and output row counts of the filter are the same after the `Filter` operator. In the query execution plan diagram, `Filter` is labeled as <span style="color:#d4628d">does not do anything</span>.
4. From a high-level view of the query execution plan on slide 2
   - the two types of metrics to be calculated (three session-based metrics and one metric for recipe views) are separated and are shown in <span style="color:blue">blue</span> and <span style="color:#df03db">dark pink</span>. This is expected since this separation was necessary since a window function function was used to find the most viewed daily recipe
     - `session_summary` and `daily_session_summary` calculate the three daily session-based metrics. These CTEs are shown in the <span style="color:blue">blue box</span>. They do not need to rank recipes based on views, so they do not need to apply the `RANK()` window function.
     - `most_viewed_recipe_daily` is the only CTE that is concerned about ranking recipes based on daily views. These CTEs are shown in the <span style="color:#df03db">dark pink box</span>.
5. From the zoomed verson of the execution plan on slide 3, the two separate daily aggregations are `LEFT JOIN`ed. The number of rows after the `JOIN` is larger than before the `JOIN` since multiple recipes are tied for most views on some days. This is labeled on slide 3 as *<span style="color:#78339a">More rows after LEFT JOIN due to ties in RANK()</span>*. It is reassuring that after concatenating ties into the same row using `LISTAGG`, the number of rows matches those in the CTE on the LHS of the `LEFT JOIN` and there is one row per date.
6. The query profile verifies findings from Exploratory Data Analysis (EDA) and are labeled on slide 3 as *<span style="color:#7a3d18">verifies finding from EDA</span>*
   - in `most_viewed_recipe_daily`, 55 rows are produced which matches 55 recipe views found during [EDA](https://nbviewer.org/github/edesz/adv-sql/blob/main/07_week3_exercise_eda.ipynb)
   - in `session_summary`, 178 rows are produced which the total unique sessions found during [EDA](https://nbviewer.org/github/edesz/adv-sql/blob/main/07_week3_exercise_eda.ipynb)
7. In the first CTE, the `WindowFunction` node (shown in the <span style="color:green">green box</span> on slide 3) is the most expensive node of the entire query. This is expected for two reasons
   - since two window functions were used and they were both applied to all events in the table. It was necessary to read all events from the table in order to calculate session length for the second report metric (average session length)
   - the window function contains a nested `PARSE_JSON`, which is also applied to each row since we need to extract the attributes of the `event_details` column for all events in the table so they could be accessed in CTEs
8. There are a lot of processing operations being performed such as aggregations, window functions and `JOIN`s. So, from the **Profile Overview**, it is not surprising that processing time dominates the overall time required to execute the query.

**From the output of the `get_query_operator_stats()` function**

1. The query does not suffer from two of the four common query problems (exploding `JOIN`s and intermediate results are too large to fit in memory) and this should help when the query has to scale from capturing metrics for 200 users in the report to capturing 5,000 users.
2. The other two common query problems were not relevant here
   - use of `UNION` when `UNION ALL` was sufficient
     - this was not applicable here since `UNION` was not used in the query
   - inefficient pruning
     - this was not applicable here since the table used does have have clustering enabled

### Structuring of query to handle higher website traffic

1. **Dropping duplicated events (duplicated `event_timestamp`s) within each session is expensive and it will become a bigger problem when website traffic increases. Dropping duplicates <u>is not necessary</u> to correctly calculate the four metrics required in the report.**
   - (question 1) count the number of unique sessions
     - only the `session_id` column is needed to count the number of unique sessions. The `session_id` is the same for all events in the session, regardless of whether some `event_timestamp`s are duplicated or not. So, duplicates were not dropped when counting the number of unique sessions.
   - (question 2) get the average session length on a daily basis
     - The `MIN()` and `MAX()` functions pick up the `event_timestamp`s of the first and last events respectively. The difference between these two timestamps is sufficient to calculate session length. The duplicates occur at the start of the session, which effects the use of the `MIN()` function. However, since `event_timestamp` is the same for the duplicated events, the `MIN()` function correctly picks up the first event of the session even if duplicated events are present in the table. So, duplicates were not dropped when calculating the average session length each day.
   - (question 3) count the number of searches performed before viewing a recipe
     - only events corresponding to views of the home page (which occurred at the start of each session) were found to be duplicated and not events that were logged later in a session when a user (a) performed a search or (b) viewed a recipe. Checking if a recipe was viewed and counting the number of searches performed before viewing the recipe do not need to take home page views into account. Filtering the `JSON` of the `event_details` column is sufficient to get (a) and (b), regardless of whether the events in each session before the user search are duplicated. So, duplicates were not dropped for calculating the average number of searches performed in each user session in which a recipe was viewed.
   - (question 4) get the ID of the most viewed recipe
     - user views of the recipe only occurs at the end of a session. So first event of the session is not needed to extract this ID. Duplicates only occurr for the first event of the session. So, duplicates were not dropped for getting the ID of the most viewed recipe.
2. **(Column Pruning or Projection Pushdown) Redundant columns are present in the `website_activity` table and are excluded in all CTEs. Including such columns in CTEs will become more expensive as website traffic increases.** This is especially important in the first CTE (in step 1. of the query) which returns output at the events level. Redundancy is handled as follows
   - the `event_timestamp` is used to calculate the average session length in seconds on a daily basis. If the `event_timestamp` column is used from the `website_activity` table then the `event_id` column is redundant. Also, the report does not require metrics at the event level. So, `event_id` is excluded from the first (`session_last_event_all_events_json`) CTE.
   - the `session_id` column is used to count the number of daily unique sessions. Each user session is assigned a unique `session_id`. Also, the report does not require a count of the number of users. So, if the `session_id` column is used then the `user_id` column is redundant and so it is also excluded from the first (`session_last_event_all_events_json`) CTE.
3. Event processing is only performed in the first CTE (`session_last_event_all_events_json`). **[Parsing `JSON` can be expensive](https://www.castordoc.com/how-to/how-to-use-parse-json-in-snowflake) and will become slower as website traffic increases, so it is only performed in this first CTE** in order to convert the `event_details` column into a `VARIANT` type. Parsing `JSON` is needed at the events level, since we need to extract information later about the type of events and order of events in each session. By calling `PARSE_JSON` in the first CTE, the `event` and `recipe_id` attributes can be accessed from the `VARIANT` column in later CTEs. No additional datatype conversion operations are performed in the overall query. All other CTEs (except the first CTE) return aggregated outputs at the session of day level. `PARSE_JSON` is called twice since it is needed for the following
   - `PARSE_JSON` is called in the `session_last_event_all_events_json` CTE to convert the `event_details` column from a `VARCHAR` into `VARIANT` datatype for every event in this column
   - `PARSE_JSON` is called inside a window function in the same CTE to convert the last event in the `event_details` column into a `VARIANT` datatype which can be later queried to access the `recipe_id`
4. String functions are expensive and will be a bigger performance hit as the number of users on Virtual Kitchen increases. One string operation is needed: to clean the `recipe_id` by removing the leading and trailing `"`. Multiple recipes might be viewed every day. To minimize the use of string functions, the most-viewed recipe(s) are first identified and then the `recipe_id` is cleaned. So, the query takes advantage of [SQL order of operations](https://builtin.com/data-science/sql-order-of-execution) to only apply string operations *after* daily aggregation (using `GROUP BY`) is performed. By doing this, **the use of string cleaning operations is minimized since only one or two `recipe_id` strings are cleaned per day.**
5. **Only the best `recipe_id`s per day are returned by the `most_viewed_recipe_daily` CTE using `QUALIFY`**. As mentioned above, multiple recipes might be viewed every day. As the platform is opened to more users, the number of recipes viewed daily might grow further. So, the number of rows returned by the `most_viewed_recipe_daily` will also grow. The report only needs the ID(s) of the most-viewed recipe per day. This was retrieved using a window function (`RANK()`). If only this ID (`RANK() OVER(...) = 1`) is returned daily, then the number of rows returned by the `most_viewed_recipe_daily` CTE is reduced. This approach has two performance benefits when extracting the most viewed recipe(s) daily
   - avoids duplication of the window logic to get the most viewed recipe(s) daily between this CTE and the final CTE (`daily_report_summary_sessions_recipe`)
   - filters out recipes as early as possible in the overall query, which improves performance when the number of users grows

## Export Query to `.sql` File

Export the report query to a standalone SQL file

In [16]:
%%writefile 08_week3_exercise_chosen_approach.sql
/* 1. perform the following operations on the events level data
  - use PARSE_JSON to convert (a) last event and (b) all events in the event_details column into the VARIANT datatype that holds JSON information whose attributes be accessed later
  - extract session date from the event_timestamp column
  - exclude columns that are not used in downstream CTEs of the overall query
    - user_id
    - event_id
2. get the most viewed recipe_id every day
  - (a) get all sessions in which the last event was viewing a recipe
  - (b) for all matching sessions from (a), use GROUP BY get the
    - (i) recipe_id
    - (ii) number of events in which a recipe was viewed per day
      - this is the number of daily views per recipe, which will be used to rank the recipes
  - (c) get the recipe_id with the most daily views
    - use a RANK() window function to assign ranks to the `recipe_id`s based on their number of daily views
    - use QUALIFY to get the recipe_ids for which RANK() OVER(...) = 1
3. Summarize each session by getting the following stats per session per date
  - (a) indicate if the session ended with a view of a recipe (i.e. if the last event in a session was to view a recipe)
  - (b) count the number of events in which a search performed during the session
    - this will be used to answer question 3
  - (c) calculate the session length in seconds using TIMESTAMPDIFF('second', ...)
4. From the output of step 3., get the following stats per date
  - (a) number of unique sessions from 3. (a)
  - (b) average session length from 3. (c)
  - (c) average number of searches performed before viewing recipe
    - for sessions in which a recipe was viewed from 3. (a), this is the average of the number of daily views during those sessions which was found in 3. (b)
5. Combine daily session summary from step 4. and most daily viewed recipe(s) from step 2.
  - (a) perform a LEFT JOIN using the session date column since both steps 2. and 4. are aggregated by date
  - (b) in the daily_session_summary CTE, session date was extracted as the date on which the session started. Some sessions can be spread out across two days.
    Such sessions should only have been counted on the first day of the session and not on the second day. Also, session length was recorded on the first date
    since session_date was recorded on the date on which the session started, so the session length should been excluded from the second date. In summary, such
    multi-day sessions should not have been (i) counted on both days and (ii) used to calculate session length on the second day. However, such logic has
    not been implemented earlier in the query. So, these rows must now be filtered out using WHERE session_length > 0.
  - (c) the output of step 2. contains some dates on which multiple recipes were tied as the most viewed recipe since they had the same number of views. For
    these rows, concatenate the recipe_ids into a comma-separated string on a single row using GROUP BY+LISTAGG. Based on SQL order of operations, the JOIN
    is executed before the GROUP BY. For this reason, a LEFT JOIN is needed so that no rows are lost if multiple recipes were tied for the most daily views.
  - (d) for readability, sort the result in chronological order using the session date column */

/* step 1. get JSON of the last event and all events performed per session */
WITH session_last_event_all_events_json AS (
    SELECT session_id,
           event_timestamp,
           /* take the start date of the session as the session date */
           MIN(TO_DATE(event_timestamp)) OVER(PARTITION BY session_id) AS session_date,
           /* get JSON of last event */
           LAST_VALUE(PARSE_JSON(event_details)) OVER(
               PARTITION BY session_id
               ORDER BY event_timestamp
           ) AS last_event_json,
           /* get JSON of all events */
           PARSE_JSON(event_details) AS event_json
    FROM events.website_activity
),
/* step 2. get the most viewed recipe each day
   (a) get sessions in which the last event was viewing a recipe
   (b) for the (last) event in the sessions from 2. (a), get (i) recipe_id and
   (ii) get the number of times the recipe was viewed daily
   (c) get the recipe_id with the most daily views using QUALIFY */
most_viewed_recipe_daily AS (
    SELECT session_date,
           /* step 2. (b) (i) get recipe_id */
           last_event_json:recipe_id AS recipe_id
    FROM session_last_event_all_events_json
    /* step 2. (a) get sessions where the last event was to view a recipe */
    WHERE last_event_json:event = 'view_recipe'
    /* step 2. (b) get events in which the recipe was viewed */
    AND event_json:event = 'view_recipe'
    GROUP BY ALL
    /* step 2. (c) get the recipe_id with the most daily views */
    QUALIFY RANK() OVER(PARTITION BY session_date ORDER BY COUNT(*) DESC) = 1
),
/* step 3. summarize each session by getting the following stats
   (a) indicate if it ended with a user viewing a recpie
   (b) count number of searches performed during the session
   (c) get session length */
session_summary AS (
    SELECT session_date,
           session_id,
           /* step 3. (a) indicate if session ended with viewing recipe */
           (
               CASE
                   WHEN last_event_json:event = 'view_recipe'
                   THEN TRUE
                   ELSE FALSE
               END
           ) AS has_viewed_recipe,
           /* step 3. (b) count number of searches performed per session */
           SUM(
               CASE
                   WHEN event_json:event = 'search'
                   THEN 1
                   ELSE 0
               END
           ) AS num_searches,
           /* step 3. (c) get session length */
           TIMESTAMPDIFF(
               'second', MIN(event_timestamp), MAX(event_timestamp)
           ) AS session_length
    FROM session_last_event_all_events_json
    GROUP BY ALL
),
/* step 4. aggregate stats per day to get the following
   (a) number of sessions
   (b) average session length in seconds
   (c) average number of searches before viewing a recipe */
daily_session_summary AS (
    SELECT session_date,
           /* step 4. (a) number of daily sessions */
           COUNT(session_id) AS num_sessions,
           /* step 4. (b) average daily session length */
           AVG(session_length) AS avg_session_length_seconds,
           /* step 4. (c) average number of searches before viewing recipe */
           AVG(
               CASE
                   WHEN has_viewed_recipe = TRUE
                   THEN num_searches
                   ELSE NULL
               END
           ) AS avg_num_searches_before_viewing_recipe
    FROM session_summary
    GROUP BY ALL
),
/* step 5. combine daily session summary and most daily viewed recipe(s)
   (a) perform LEFT JOIN between recipe(s) with the most daily views and
   daily session summary
   (b) handle multi-day sessions using WHERE avg_session_length_seconds > 0
   (c) concatenate most-viewed recipes into comma-delimited string, using
   GROUP BY+LISTAGG
   (d) sort result by session date in chronological order for readability */
daily_report_summary_sessions_recipe AS (
    SELECT * EXCLUDE(most_viewed_recipe),
           /* clean the most_viewed_recipe ID(s) per day */
           REPLACE(most_viewed_recipe, '"', '') AS most_viewed_recipe
    FROM (
        SELECT ds.session_date,
               ds.num_sessions,
               ds.avg_session_length_seconds,
               ds.avg_num_searches_before_viewing_recipe,
               /* step 5. (b) concatenate ties beteween most-viewed recipes into
                  single row */
               LISTAGG(dv.recipe_id, ', ') AS most_viewed_recipe
        FROM daily_session_summary AS ds
        /* step 5. (a) combine session summary and most-viewed receipe each day */
        LEFT JOIN most_viewed_recipe_daily AS dv USING (session_date)
        /* step 5. (b) exclude sessions which have been counted on previous day and so
           have a session length of zero */
        WHERE avg_session_length_seconds > 0
        GROUP BY ALL
        /* step 5. (c) sort result in chronological order for readability */
        ORDER BY ds.session_date
    )
)
SELECT *
FROM daily_report_summary_sessions_recipe

Writing 08_week3_exercise_chosen_approach.sql


## Disconnect

In [17]:
%sql --close connection